I struggled with using Cloud GPU instances quite a bit and found no good guide (even in the documentation on the respective websites, which was somewhat helpful but often presupposed some knowledge on my part). I didn't even know what was possible with these cloud instances. I had experimented with Paperspace Gradient but found it ultimately unsatisfying because it used outdated versions of PyTorch and other libraries, because there was a limit to the number of notebooks one could create and because the interface was buggy and sometimes difficult to use. So I turned to Cloud GPU instances offered by sites such as Lambda Labs, Vasi.ai and JarvisLabs, which I thought could offer me better freedom. 

When I first started with Lambda Labs, I watched their tutorial and learned one could access Jupyter Labs and an Ubuntu terminal with an instance on their website, but I failed to see how this was an improvement over Paperspace Gradient. What I wanted was an interface where I could download datasets locally, use my IDE to create and test models and then use the GPU to accelerate the training of those models. After researching these instances, I found that it IS possible to do so, and that's what I'm going to show today. 

**Step One: SSH Key Set-Up**

The first thing you'll need to do is create an SSH key on your local device. This is how you'll connect to the GPU instance after you've initialized it. We'll create this key using the command line. On Mac, open terminal and enter:

In [ ]:
!ssh-keygen -t rsa -b 4096 -C "your-email@email.com" 

This will create the SSH Key that we need. The computer will ask you to name a file to save the keys to. I don't fully understand the implications of naming a file, so I stick with the default and just press enter. The computer will ask you to input a password and then it will create the file and save it in your home directory. It will tell you the location, so don't worry about losing it. 

After we've done this, we need to navigate to that directory and input the public key into Lambda Labs. To do this, we'll navigate to the home directory and look for a folder titled ".ssh". On Mac, this is usually hidden, so hit SHIFT+CMD+. to unhide the folders. Within the .ssh folder there should be two documents. Open both using TextEdit or a similar app. One will say "Private Key" and the other will say "Public Key". On Lambda Labs, click "SSH Keys" on the main menu and click "Add SSH Key". Then copy and paste the public key into the text box and give it a nickname. 

This completes the initial set up.

**Step Two: Prepare your files for export**

Once the cloud GPU instance is running, you'll be charged by the minute, so you want to have everything ready before you launch the instance. If you're reading this, you're probably a beginner and if you're a beginner you've probably created most of your models so far using Jupyter Notebooks, which let you run code one piece at a time. If you're going to use a cloud GPU instance, you'll need to get used to running code using the command line and creating projects to be saved as Python files instead of Notebooks. If this sounds hard, don't worry, it's not as hard as it sounds. I would recommend doing the following. 

Using PyCharm or some other python IDE, create a new project. Have a main.py file, which will have your DataLoaders and training loops. Create separate scripts that download and preprocess the data you're going to use, this way the cloud GPU won't waste time with those processes. Create a separate script with your implementations of any special classes, such as Datasets or Models if you're using PyTorch. Then, in your main.py, import those files so main.py has access to those classes and any other helper functions. 

Importing other .py files in python scripts is pretty easy. For example, if you have a file called model.py which includes your custom model architecture, you can import it to your main with simply:

In [ ]:
from model import *

Note, both files must be in the same directory. It is not necessary to put ".py" in the import statement.

When you're ready to test your project, you can run it from the command line with:

In [ ]:
!python main.py

If your data is saved locally, the next thing you'll want to do is compress everything so it'll be easier and faster to upload to the instance once it's launched. You can do this with the following command:

In [ ]:
!tar -czvf NAME.tar.gz "/Directory/You/Want/To/Compress"

#If you want to specify a destination directory for the compressed file: 
!tar -czvf /path/to/destination/NAME.tar.gz "/Directory/You/Want/To/Compress"

**Step Three: Launch the instance and connect using SSH**

Now we're ready to launch the instance. I'll leave out the mechanics of doing so, as it'll vary from website to website and they tend to make it pretty easy. The more difficult part is connecting via SSH. Once the instance is launched, you should get either an SSH username or an instance IP from whichever website you're using. To connect, input the following command on your local terminal:

In [ ]:
!ssh -i ~/path/to/private/key ubuntu@your_instance_ip

Note: Your instance may not be using ubuntu. It is the default for lambda labs but other providers use different linux environments for their servers. 
The computer may ask "This key is not known by any other names.
Are you sure you want to continue connecting (yes/no/[fingerprint])?" Enter yes. You will then be prompted to enter the password you used when you created the SSH key initially. 

If everything worked, your computer will connect and the prompt string of your terminal will change to reflect the new server. 

**Step Four: Transferring your files**

Now that we've connect, the final step is to transfer our files to the remote instance. We do this by entering the following command on our local terminal:

In [ ]:
!scp -r /path/to/local/directory username@remote_host:/path/to/remote/directory

Note: the -r argument makes the transfer "recursive" meaning all files and subdirectories are transferred as well. 

**Troubleshooting**

I ran into two main issues while experimenting with this. 

1. Permission denied when connecting via SSH. This happened the first time because I set up my SSH Key incorrectly. The second time, I was able to fix the issue by running:

In [ ]:
!chmod 400 /path/to/local/directory

2. If you have any directory paths hard coded in your code, those paths will no longer be valid. I've yet to find an elegant solution to this problem, so I simply change the hard coding when I upload the files and change it back when I'm ready to test locally again. 